In [ ]:
pip install timm==0.8.21.dev0

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, Subset
from torchvision.datasets import ImageFolder
from sklearn.model_selection import train_test_split
import timm
import numpy as np

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Data transformations
data_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [3]:
# Load the dataset
data_path = '/kaggle/input/hackathon-online-thai-silk-pattern-recognition/Train/Train'
dataset = ImageFolder(data_path, transform=data_transforms)

# Split the dataset into training and validation sets
train_size = 0.8
indices = list(range(len(dataset)))
train_indices, val_indices = train_test_split(indices, test_size=1-train_size, stratify=dataset.targets, random_state=42)

train_data = Subset(dataset, train_indices)
val_data = Subset(dataset, val_indices)

# Create data loaders for training and validation
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_data, batch_size=32, shuffle=True, num_workers=4)

/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [7]:
class CutMix:
    def __init__(self, dataloader, alpha=1.0):
        self.dataloader = dataloader
        self.alpha = alpha

    def __iter__(self):
        for inputs, labels in self.dataloader:
            batch_size = inputs.size(0)
            indices = torch.randperm(batch_size)
            inputs_ = inputs.clone()
            labels_ = labels.clone()

            # Sample the mixing ratios from a beta distribution
            lam = np.random.beta(self.alpha, self.alpha)

            # Create a random bounding box
            cut_rat = np.sqrt(1.0 - lam)
            cut_w = (inputs.size(2) * cut_rat).astype(int)
            cut_h = (inputs.size(3) * cut_rat).astype(int)
            cx = np.random.randint(0, inputs.size(2) - cut_w)
            cy = np.random.randint(0, inputs.size(3) - cut_h)

            # Apply CutMix
            inputs_[:, :, cx:cx + cut_w, cy:cy + cut_h] = inputs[indices, :, cx:cx + cut_w, cy:cy + cut_h]
            lam = 1.0 - cut_w * cut_h / (inputs.size(2) * inputs.size(3))
            labels = (labels, labels_[indices], lam)

            yield inputs_, labels

    def __len__(self):
        return len(self.dataloader)

In [8]:
train_loader = CutMix(train_loader, alpha=1.0)

# Load the EVA model
model = timm.create_model('eca_nfnet_l0', pretrained=True)

# Modify the last layer to match the number of classes
num_classes = 4
model.head.fc = nn.Linear(model.head.fc.in_features, num_classes)

# Set up the device, loss function, and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 15

In [11]:
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for inputs, labels_tuple in train_loader:
        inputs = inputs.to(device)
        labels, labels2, lam = labels_tuple
        labels, labels2 = labels.to(device), labels2.to(device)

        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels) * lam + criterion(outputs, labels2) * (1 - lam)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_loss /= len(train_loader)

    # Validation loop
    model.eval()
    val_loss = 0.0
    val_acc = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            val_acc += torch.sum(preds == labels).item()

    val_loss /= len(val_loader)
    val_acc /= len(val_data)

    print(f'Epoch: {epoch + 1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

KeyboardInterrupt: 

In [17]:
# Save the trained model
model_save_path = "trained_model.pt"
torch.save(model,model_save_path)

In [19]:
model = torch.load('/kaggle/working/trained_model.pt')
model.eval()

AttributeError: 'collections.OrderedDict' object has no attribute 'eval'

In [14]:
import os
import pandas as pd
import torch
import torchvision.transforms as transforms
from PIL import Image

# Load model
# model = torch.load("/kaggle/working/trained_model.pth")
# model.eval()

# Load test data
test_dir = "/kaggle/input/hackathon-online-thai-silk-pattern-recognition/Test/Test"
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Make predictions
predictions = []
for filename in os.listdir(test_dir):
    image_path = os.path.join(test_dir, filename)
    image = Image.open(image_path).convert("RGB")
    input = test_transform(image).unsqueeze(0).to("cuda")
    with torch.no_grad():
        output = model(input)
        _, predicted_label = torch.max(output, 1)
        predictions.append(predicted_label.item())

TypeError: 'collections.OrderedDict' object is not callable

In [ ]:
# Create submission file
submission_df = pd.DataFrame({"Id": os.listdir(test_dir), "Predicted": predictions})
submission_df.to_csv("submission.csv", index=False)